<a href="https://colab.research.google.com/github/ktichola/Image-matching/blob/main/Image_Matching_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-matching-challenge-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71885%2F8143495%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073243Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5f36085f84a8e7f25aecb5d040c82f153ac36e83a651e60eba4b9ddb7f73ad1e40d445ebff92d785f9caf31949535f5f0f78ea7dce2cf3c3a3b8cfeb567d650f539ce5c56a2f419004e1c92f6cdc810e9b943409c5b038e28323ec9734b001b95021449d848ff42cf93139a8c466d5583696f786303778353f56f34dd941801358a71b0cb98ea4af7d023efcb442a7d56fb63d9b75a5be5ee5a1d56b357b72d59de33de244ca3510b4c57295e7a23c3728a1e2e25bf08c1cca37c03b2a21761e4241b0ebb0ade848f92cbb81587d038f555778ca64631c7e62ca6af57910a280ed4b170c284b3c1bfbf0d0b56953a0ff7d2990b245ae4a1e124d20dcc0c3098b,imc2024-packages-lightglue-rerun-kornia:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628051%2F7884485%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073243Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9e3a9c43a814d226ce3e743cfa6bf924029a4b17bb47f5e1b0327e38d39190111cee6f192ac4e3c7e49023cbdcb371e2bc6dadaefe81af2e7aea0cf024213e008188db1fcd612da911296fa0f6fa2fd752c6784ef092d59886e0bb1556de59ce31e98db4b995d1a0fbfe7d1b8b1ba5e8b57a273a87b6bf08f1ea8fc2e6cef3249fa75061ee61bc6bd5d3f0f4b55f063a7cc4f61ee69a43ba01c1bc42bda5beff6ad0052115507a386c4483799173caa2849790c4e243632fcce440eef0ca84a5e328dbdec90b7f29d3d0a8b62ceb2ce815b089cb490485d9939f9360a5552f6ceac1be2747edb3d2f1a9e279087ea5a47ff21b68a5aa0253ef523f5e51b402ec,colmap-db-import:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628331%2F7884725%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073243Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D86408ca547f556ce10acfe8a095371f395845a02dc57e0e1bc027b74d55b04dd7cee489286844af1b0c85d0eb73fbd72b9e80b93e39b2a2e52057464cdf9dacd50f71e3bc16dee03b2c165cbb75ac06b15e87b4758aceaac6d6d9d51ec7966d5f0b5be60db58212f61e79bfd198db9c87a6e3868892845c718c96cbf9180955afa92eb6c3f3ad1ffe01d6663ab7826522a138459e2e816b04b267649ea0618cda27824959808c7fea58830695efe08223630176df43b945bf5f151e48d315e8aea22ba2f9ebaaae8ec053cf91f35a7940d1134be61e0d3ac2958a70f7a9ff831e43bc6c34aaf17a599f8c7faebc4e05ca796020ccd25df04fd903997bbca0eb3,dinov2/pytorch/base/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3326%2F4534%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073243Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7615615e9cf81ee93f0f1c70b484238b209cfd7042e975cd5492e359fb15ef872d2ae9dfed3398e034dc765e23027a51659e39e24cd689de700aabfb4014aa276ca7d4997d0a1f2350200d4c1f9ea475da28be8125526143731a8e2e391c63eb3a6394a54b7331ee05102f275f547cb4ae80c6fec405f149daa68db2a29bdcc099954b758aba80f88a3b25d22a90ce40f1bdd136c307a6cbf66440640b6f6e81c6bc2de92f420d5e61ed1b351ca4a0de1ec01daba4d957ef8045ab317118b58eb48019802d2e61ba88b905b5ab9db947d24e028139e86dd8a72489f20506e58302a287d40f5664b89180baca2b56063755e3ac85b16c9e3231d8c72e5d56db6c,lightglue/pytorch/aliked/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14317%2F17191%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073243Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D90c9b04232741934f049aca731d63282a3e60ced0e20980451224fc0af3a70cd330653ecd69c60ba796cd03f131bfe29055e3541d47e5424ca188f02e0fb757e84bf26b585f4d4c945d1934ed1ed28fa8414d09df2da8ac9b6384db8180d5f61a17ee89bc804da4e2a036532dcfa9e5e95729f8f9c2fa32c75dda402dac162db1ef08daa62a197a18ee0645b28ff289468e379107595fe911fe62ff3351d864499dabe939f8327f72b3c6e61ba3543412193ad52eeaf198419fb32f3a1747a962f543e9851fe3404a9caeb856d689228170b54e628ab7430269a0be68d015afb0e2bebb67ff205ea89b3d91b72cf9742af2f0efc00747e91650fba47452432a7,aliked/pytorch/aliked-n16/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14611%2F17555%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T073243Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0fd2e6938b12d91872a35b23fada60dfcdc9afa03be431cd0a3980cfc17288851cb495f1689b1bddc0afa4a5de0709cd4fe1c789a430bb84601dde14f83c63323e05356029721231387074eb54299803d980dae59da3c36a353798e0506dd5050ab408d4c06ef0654ba315ad7821373dc285a05d2a9d9891ff83e1ac7ee9bc0bd60a19504b4978b5194f8954219d7e9f3ef2ea7a9df396b34cf7d769c8da27f7826ab0e2b2fc71f5b3568b1e75b6155864b4eaca5e5ea5a8921da509576e347d5e9fa3fe8cc8a7b2c95b1808548c4cf4e14195e6cc103166a6d4fa8301003597b259791511cdffbb56187aed9da163d1e0b0714f1f152843c36ee7e201b83e2e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

In [ ]:
import matplotlib.pyplot as plt

import os
from tqdm import tqdm
from pathlib import Path
from time import time, sleep
from fastprogress import progress_bar
import gc
import numpy as np
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from typing import Any
import itertools
import pandas as pd

# CV/MLe
import cv2
import torch
from torch import Tensor as T
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image
from transformers import AutoImageProcessor, AutoModel

import torch
from lightglue import match_pair
from lightglue import LightGlue, ALIKED
from lightglue.utils import load_image, rbd

# 3D reconstruction
import pycolmap

# Data importing into colmap
import sys
sys.path.append("/kaggle/input/colmap-db-import")

# Provided by organizers
from database import *
from h5_to_db import *

def arr_to_str(a):
    """Returns ;-separated string representing the input"""
    return ";".join([str(x) for x in a.reshape(-1)])

def load_torch_image(file_name: Path | str, device=torch.device("cpu")):
    """Loads an image and adds batch dimension"""
    img = K.io.load_image(file_name, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img


def embed_images(
    paths: list[Path],
    model_name: str,
    device: torch.device = torch.device("cpu"),
) :
    """Computes image embeddings.

    Returns a tensor of shape [len(filenames), output_dim]
    """
    processor = AutoImageProcessor.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).eval().to(device)

    embeddings = []

    for i, path in tqdm(enumerate(paths), desc="Global descriptors"):
        image = load_torch_image(path)

        with torch.inference_mode():
            inputs = processor(images=image, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs) # last_hidden_state and pooled

            # Max pooling over all the hidden states but the first (starting token)
            # To obtain a tensor of shape [1, output_dim]
            # We normalize so that distances are computed in a better fashion later
            embedding = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=-1, p=2)

        embeddings.append(embedding.detach().cpu())
    return torch.cat(embeddings, dim=0)


def load_torch_image(file_name: Path | str, device=torch.device("cpu")):
    """Loads an image and adds batch dimension"""
    img = K.io.load_image(file_name, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

def get_image_pairs(paths: list[Path],model_name: str,similarity_threshold: float = 0.6,
                    tolerance: int = 1000,min_matches: int = 20,exhaustive_if_less: int = 20,
                    p: float = 2.0,device: torch.device = torch.device("cpu")):
    # We try to minimize the camera to reconstruct the scene due to saving computing cost and minimizing the error.
    if len(paths) <= exhaustive_if_less:
        return get_pairs_exhaustive(paths)
    matches = []
    embeddings = embed_images(paths, model_name)
    distances = torch.cdist(embeddings, embeddings, p=p)

    # Remove pairs above similarity threshold (if enough)
    mask = distances <= similarity_threshold
    image_indices = np.arange(len(paths))

    for current_image_index in range(len(paths)):
        mask_row = mask[current_image_index]
        indices_to_match = image_indices[mask_row]

        # We don't have enough matches below the threshold, we pick most similar ones
        if len(indices_to_match) < min_matches:
            indices_to_match = np.argsort(distances[current_image_index])[:min_matches]

        for other_image_index in indices_to_match:
            # Skip an image matching itself
            if other_image_index == current_image_index:
                continue

            # We need to check if we are below a certain distance tolerance
            # since for images that don't have enough matches, we picked
            # the most similar ones (which could all still be very different
            # to the image we are analyzing)
            if distances[current_image_index, other_image_index] < tolerance:
                # Add the pair in a sorted manner to avoid redundancy
                matches.append(tuple(sorted((current_image_index, other_image_index.item()))))

    return sorted(list(set(matches)))



In [ ]:
def detect_keypoints(
    paths: list[Path],
    feature_dir: Path,
    num_features: int = 4096,
    resize_to: int = 1024,
    device: torch.device = torch.device("cpu"),
) -> None:
    """Detects the keypoints in a list of images with ALIKED

    Stores them in feature_dir/keypoints.h5 and feature_dir/descriptors.h5
    to be used later with LightGlue
    """
    dtype = torch.float32 # ALIKED has issues with float16

    extractor = ALIKED(
        max_num_keypoints=num_features,
        detection_threshold=0.01,
        resize=resize_to
    ).eval().to(device, dtype)

    feature_dir.mkdir(parents=True, exist_ok=True)

    with h5py.File(feature_dir / "keypoints.h5", mode="w") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="w") as f_descriptors:

        for path in tqdm(paths, desc="Computing keypoints"):
            key = path.name

            with torch.inference_mode():
                image = load_torch_image(path, device=device).to(dtype)
                features = extractor.extract(image)

                f_keypoints[key] = features["keypoints"].squeeze().detach().cpu().numpy()
                f_descriptors[key] = features["descriptors"].squeeze().detach().cpu().numpy()

In [ ]:
def keypoint_distances(
    paths: list[Path],
    index_pairs: list[tuple[int, int]],
    feature_dir: Path,
    min_matches: int = 15,
    verbose: bool = True,
    device: torch.device = torch.device("cpu"),
) -> None:
    """Computes distances between keypoints of images.

    Stores output at feature_dir/matches.h5
    """

    matcher_params = {
        "width_confidence": -1,
        "depth_confidence": -1,
        "mp": True if 'cuda' in str(device) else False,
    }
    matcher = KF.LightGlueMatcher("aliked", matcher_params).eval().to(device)

    with h5py.File(feature_dir / "keypoints.h5", mode="r") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="r") as f_descriptors, \
         h5py.File(feature_dir / "matches.h5", mode="w") as f_matches:

            for idx1, idx2 in tqdm(index_pairs, desc="Computing keypoing distances"):
                key1, key2 = paths[idx1].name, paths[idx2].name

                keypoints1 = torch.from_numpy(f_keypoints[key1][...]).to(device)
                keypoints2 = torch.from_numpy(f_keypoints[key2][...]).to(device)
                descriptors1 = torch.from_numpy(f_descriptors[key1][...]).to(device)
                descriptors2 = torch.from_numpy(f_descriptors[key2][...]).to(device)

                with torch.inference_mode():
                    distances, indices = matcher(
                        descriptors1,
                        descriptors2,
                        KF.laf_from_center_scale_ori(keypoints1[None]),
                        KF.laf_from_center_scale_ori(keypoints2[None]),
                    )

                # We have matches to consider
                n_matches = len(indices)
                if n_matches:
                    if verbose:
                        print(f"{key1}-{key2}: {n_matches} matches")
                # Store the matches in the group of one image
                group  = f_matches.require_group(key1)
                if n_matches >= min_matches:
                     group.create_dataset(key2, data=indices.detach().cpu().numpy().reshape(-1, 2))

In [ ]:
class Config:
    base_path: Path = Path("/kaggle/input/image-matching-challenge-2024")
    feature_dir: Path = Path.cwd() / ".feature_outputs"
    device: torch.device= K.utils.get_cuda_device_if_available(0)
    pair_matching_args = {
        "model_name": "/kaggle/input/dinov2/pytorch/base/1",
        "similarity_threshold": 0.3,
        "tolerance": 1000,
        "min_matches": 20,
        "exhaustive_if_less": 20,
        "p": 2.0
    }
    keypoint_detection_args = {
        "num_features": 4096,
        "resize_to": 1024,
    }

    keypoint_distances_args = {
        "min_matches": 15,
        "verbose": False,
    }
    colmap_mapper_options = {
        "min_model_size": 3, # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
        "max_num_models": 2,
    }

In [ ]:
def parse_sample_submission(
    base_path: Path,
) -> dict[dict[str, list[Path]]]:
    """Construct a dict describing the test data as

    {"dataset": {"scene": [<image paths>]}}
    """
    data_dict = {}
    with open(base_path / "sample_submission.csv", "r") as f:
        for i, l in enumerate(f):
            # Skip header
            if i == 0:
                print("header:", l)

            if l and i > 0:
                image_path, dataset, scene, _, _ = l.strip().split(',')
                if dataset not in data_dict:
                    data_dict[dataset] = {}
                if scene not in data_dict[dataset]:
                    data_dict[dataset][scene] = []
                data_dict[dataset][scene].append(Path(base_path / image_path))

    for dataset in data_dict:
        for scene in data_dict[dataset]:
            print(f"{dataset} / {scene} -> {len(data_dict[dataset][scene])} images")

    return data_dict

In [ ]:
def create_submission(
    results: dict,
    data_dict: dict[dict[str, list[Path]]],
    base_path: Path,
) -> None:
    """Prepares a submission file."""

    with open("submission.csv", "w") as f:
        f.write("image_path,dataset,scene,rotation_matrix,translation_vector\n")

        for dataset in data_dict:
            # Only write results for datasets with images that have results
            if dataset in results:
                res = results[dataset]
            else:
                res = {}

            # Same for scenes
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}

                # Write the row with rotation and translation matrices
                for image in data_dict[dataset][scene]:
                    if image in scene_res:
                        print(image)
                        R = scene_res[image]["R"].reshape(-1)
                        T = scene_res[image]["t"].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    image_path = str(image.relative_to(base_path))
                    f.write(f"{image_path},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n")

In [ ]:
results = {}
config = Config()
data_dict = parse_sample_submission(config.base_path)
datasets = list(data_dict.keys())


for dataset in datasets:
    if dataset not in results:
        results[dataset] = {}

    for scene in data_dict[dataset]:
        images_dir = data_dict[dataset][scene][0].parent
        results[dataset][scene] = {}
        image_paths = data_dict[dataset][scene]
        print (f"Got {len(image_paths)} images")

        try:
            feature_dir = config.feature_dir / f"{dataset}_{scene}"
            feature_dir.mkdir(parents=True, exist_ok=True)
            database_path = feature_dir / "colmap.db"
            if database_path.exists():
                database_path.unlink()

            # 1. Get the pairs of images that are somewhat similar
            index_pairs = get_image_pairs(
                image_paths,
                **config.pair_matching_args,
                device=config.device,
            )
            gc.collect()

            # 2. Detect keypoints of all images
            detect_keypoints(
                image_paths,
                feature_dir,
                **config.keypoint_detection_args,
                device=config.device,
            )
            gc.collect()

            # 3. Match  keypoints of pairs of similar images
            keypoint_distances(
                image_paths,
                index_pairs,
                feature_dir,
                **config.keypoint_distances_args,
                device=config.device,
            )
            gc.collect()

            sleep(1)

            # 4.1. Import keypoint distances of matches into colmap for RANSAC
            import_into_colmap(
                images_dir,
                feature_dir,
                database_path,
            )

            output_path = feature_dir / "colmap_rec_aliked"
            output_path.mkdir(parents=True, exist_ok=True)

            # 4.2. Compute RANSAC (detect match outliers)
            # By doing it exhaustively we guarantee we will find the best possible configuration
            pycolmap.match_exhaustive(database_path)

            mapper_options = pycolmap.IncrementalPipelineOptions(**config.colmap_mapper_options)

            # 5.1 Incrementally start reconstructing the scene (sparse reconstruction)
            # The process starts from a random pair of images and is incrementally extended by
            # registering new images and triangulating new points.
            maps = pycolmap.incremental_mapping(
                database_path=database_path,
                image_path=images_dir,
                output_path=output_path,
                options=mapper_options,
            )

            print(maps)
            clear_output(wait=False)

            # 5.2. Look for the best reconstruction: The incremental mapping offered by
            # pycolmap attempts to reconstruct multiple models, we must pick the best one
            images_registered  = 0
            best_idx = None

            print ("Looking for the best reconstruction")

            if isinstance(maps, dict):
                for idx1, rec in maps.items():
                    print(idx1, rec.summary())
                    try:
                        if len(rec.images) > images_registered:
                            images_registered = len(rec.images)
                            best_idx = idx1
                    except Exception:
                        continue

            # Parse the reconstruction object to get the rotation matrix and translation vector
            # obtained for each image in the reconstruction
            if best_idx is not None:
                for k, im in maps[best_idx].images.items():
                    key = config.base_path / "test" / scene / "images" / im.name
                    results[dataset][scene][key] = {}
                    results[dataset][scene][key]["R"] = deepcopy(im.cam_from_world.rotation.matrix())
                    results[dataset][scene][key]["t"] = deepcopy(np.array(im.cam_from_world.translation))

            print(f"Registered: {dataset} / {scene} -> {len(results[dataset][scene])} images")
            print(f"Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images")
            create_submission(results, data_dict, config.base_path)
            gc.collect()

        except Exception as e:
            print(e)